In [1]:
# import simulation stuff
import conf_solo12_trot_step_adjustment_full_kinematics as conf
from robot_properties_solo.solo12wrapper import Solo12Config
from robot_properties_solo.solo12wrapper import Solo12Robot
from bullet_utils.env import BulletEnvWithGround
from simulate_solo import Simulator

# initialize robot in pybullet
env = BulletEnvWithGround()
simulator = Simulator(env, Solo12Robot(), conf)
robot = Solo12Config.buildRobotWrapper()
q0, dq0 = simulator.q0, simulator.nu0
simulator.robot.reset_state(q0, dq0) 
simulator.robot.pin_robot.framesForwardKinematics(q0)

pybullet build time: May 20 2022 19:44:17


Adding Body
"base_link" connected to "world" through joint "floating_base_joint"
joint type: joint FreeFlyer
joint placement:
  R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0

body info: 
  mass: 1.16115
  lever: 0 0 0
  inertia elements (Ixx,Iyx,Iyy,Izx,Izy,Izz): 0.00578574          0  0.0193811          0          0  0.0247612

Adding Body
"FL_SHOULDER" connected to "base_link" through joint "FL_HAA"
joint type: joint REVOLUTE with axis
joint placement:
  R =
1 0 0
0 1 0
0 0 1
  p = 0.1946 0.0875      0

body info: 
  mass: 0.148538
  lever: -0.078707      0.01         0
  inertia elements (Ixx,Iyx,Iyy,Izx,Izy,Izz):  3.024e-05  4.671e-05 0.00041193          0          0 0.00041107

Adding Body
"FL_UPPER_LEG" connected to "FL_SHOULDER" through joint "FL_HFE"
joint type: joint REVOLUTE with axis
joint placement:
  R =
1 0 0
0 1 0
0 0 1
  p =     0 0.014     0

body info: 
  mass: 0.148538
  lever: 1.377e-05 0.0193585 -0.078707
  inertia elements (Ixx,Iyx,Iyy,Izx,Izy,Izz): 0.00041107          0 0.00

In [ ]:
import pinocchio as pin
import numpy as np
rmodel = conf.rmodel 
rdata = rmodel.createData()
pin.framesForwardKinematics(rmodel, rdata, q0)
tau_gravity = pin.rnea(rmodel, rdata, q0, dq0, np.zeros_like(dq0))
force_per_leg = np.array([0., 0., (conf.robot_mass*9.81)/4.0])
Kp = 2*np.eye(12)
Kd = 0.01*np.eye(12)
Jtf = 0.
for frame_name in conf.ee_frame_names:
    J = pin.computeFrameJacobian(
        rmodel, rdata, q0, rmodel.getFrameId(frame_name), pin.LOCAL_WORLD_ALIGNED
    )
    Jtf += J.T[:, :3] @ force_per_leg
while (True):
    q_meas, qdot_meas = simulator.robot.get_state()
    tau = tau_gravity[6:] - Jtf[6:] +\
         Kp @ (q0[7:] - q_meas[7:]) + \
         Kd @ (np.zeros(12) -  qdot_meas[6:])
    simulator.robot.send_joint_command(tau) 
    simulator.env.step(sleep=True)  
   

    